# Assignment - Segmenting and Clustering Neighborhoods in Toronto

In [63]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

import csv

Libraries imported.


## An Overview of Toronto and its Geography

In [64]:
dfToro = pd.read_csv('ToroGeo.csv')

In [65]:
dfToro.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [66]:
print('The dataframe has {} boroughs and {} neighborhoods grouped by postal codes.'.format(
        len(dfToro['Borough'].unique()),
        dfToro.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods grouped by postal codes.


In [67]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="toro_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto is 43.653963, -79.387207.


In [68]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(dfToro['Latitude'], dfToro['Longitude'], dfToro['Borough'], dfToro['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [70]:
East_Toronto = dfToro[dfToro['Borough'].str.contains("East Toronto")].reset_index(drop=True)
East_Toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558


In [71]:
# Set Latitude and Longitude for East Toronto manually with good approximation
latitude = 43.66
longitude = -79.32

# create map of Manhattan using latitude and longitude values
map_East_Toronto = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, label in zip(East_Toronto['Latitude'], East_Toronto['Longitude'], East_Toronto['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_East_Toronto) 

map_East_Toronto

#### Define Foursquare Credentials and Version

In [72]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


### Let's explore Studio District of East_Toronto Borough in the dataframe.

Get the neighborhood's name.

In [73]:
East_Toronto.loc[3, 'Neighbourhood']

'Studio District'

Get the neighborhood's latitude and longitude values.

In [74]:
neighborhood_latitude = East_Toronto.loc[3, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = East_Toronto.loc[3, 'Longitude'] # neighborhood longitude value

neighborhood_name = East_Toronto.loc[3, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Studio District are 43.6595255, -79.340923.


### Find top venues that are in Studio District within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [75]:
# type your answer here

CLIENT_ID='SSXUP3QXWQI3VDJK3N0ZOLKS0DVXHDY4JEOZCHFOK0LAOYJ4'
CLIENT_SECRET='MXNMSOYUXXSX4BAW4KAUVLWTENRJ5AR45QNFEYIUXRFBFLBR'
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=SSXUP3QXWQI3VDJK3N0ZOLKS0DVXHDY4JEOZCHFOK0LAOYJ4&client_secret=MXNMSOYUXXSX4BAW4KAUVLWTENRJ5AR45QNFEYIUXRFBFLBR&v=20180605&ll=43.6595255,-79.340923&radius=500&limit=100'

Send the GET request and examine the resutls

In [76]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cdd66a71ed21914c1eebe0b'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Leslieville',
  'headerFullLocation': 'Leslieville, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 37,
  'suggestedBounds': {'ne': {'lat': 43.6640255045, 'lng': -79.33471445573701},
   'sw': {'lat': 43.6550254955, 'lng': -79.347131544263}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ad7e958f964a520001021e3',
       'name': "Ed's Real Scoop",
       'location': {'address': '920 Queen St. E',
        'crossStreet': 'btwn Logan Ave. & Morse St.',
        'lat': 43.660655832455014,
        'lng': -79.3420187548006,
        'labeledLatLngs': 

In [78]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [79]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Ed's Real Scoop,Ice Cream Shop,43.660656,-79.342019
1,Leslieville Pumps,Sandwich Place,43.660892,-79.340626
2,Queen Books,Bookstore,43.660651,-79.342267
3,Te Aro,Coffee Shop,43.661373,-79.338577
4,Hooked,Fish Market,43.660407,-79.343257
5,Purple Penguin Cafe,Café,43.660501,-79.342565
6,Leslieville,Neighborhood,43.662070,-79.337856
7,Mean Bao,Chinese Restaurant,43.661417,-79.340209
8,eastside social,Comfort Food Restaurant,43.661289,-79.339155
9,Brick Street Breads,Bakery,43.660685,-79.342501


And how many venues were returned by Foursquare?

In [80]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

37 venues were returned by Foursquare.


## Explore East Toronoto's Neighbourhoods

### Over view of venues nearby Studio District

In [81]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [82]:
# type your answer here

East_Toronto_venues = getNearbyVenues(names=East_Toronto['Neighbourhood'],
                                   latitudes=East_Toronto['Latitude'],
                                   longitudes=East_Toronto['Longitude']
                                  )


The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Business Reply Mail Processing Centre 969 Eastern


In [83]:
print(East_Toronto_venues.shape)
East_Toronto_venues.head()

(124, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676357,-79.293031,St-Denis Studios Inc.,43.675031,-79.288022,Music Venue
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West,Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [84]:
East_Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
Studio District,37,37,37,37,37,37
The Beaches,4,4,4,4,4,4
"The Beaches West,India Bazaar",22,22,22,22,22,22
"The Danforth West,Riverdale",44,44,44,44,44,44


In [85]:
print('There are {} uniques categories.'.format(len(East_Toronto_venues['Venue Category'].unique())))

There are 68 uniques categories.


## Analyze each Neighbourhoods in East Toronoto

In [86]:
# one hot encoding
East_Toronto_onehot = pd.get_dummies(East_Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
East_Toronto_onehot['Neighborhood'] = East_Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [East_Toronto_onehot.columns[-1]] + list(East_Toronto_onehot.columns[:-1])
manhattan_onehot = East_Toronto_onehot[fixed_columns]

East_Toronto_onehot.head()

,American Restaurant,Auto Workshop,Bakery,Bank,Bar,Bookstore,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Coworking Space,Dessert Shop,Diner,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food & Drink Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Latin American Restaurant,Light Rail Station,Liquor Store,Middle Eastern Restaurant,Movie Theater,Music Store,Music Venue,Neighborhood,Park,Pet Store,Pizza Place,Pub,Recording Studio,Restaurant,Sandwich Place,Seafood Restaurant,Skate Park,Smoke Shop,Spa,Sports Bar,Stationery Store,Steakhouse,Sushi Restaurant,Trail,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"The Danforth West,Riverdale",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [87]:
East_Toronto_onehot.shape

(124, 68)

#### Next, group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [88]:
East_Toronto_grouped = East_Toronto_onehot.groupby('Neighborhood').mean().reset_index()
East_Toronto_grouped

,Neighborhood,American Restaurant,Auto Workshop,Bakery,Bank,Bar,Bookstore,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Coworking Space,Dessert Shop,Diner,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food & Drink Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Latin American Restaurant,Light Rail Station,Liquor Store,Middle Eastern Restaurant,Movie Theater,Music Store,Music Venue,Park,Pet Store,Pizza Place,Pub,Recording Studio,Restaurant,Sandwich Place,Seafood Restaurant,Skate Park,Smoke Shop,Spa,Sports Bar,Stationery Store,Steakhouse,Sushi Restaurant,Trail,Yoga Studio
0,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.117647,0.000000,0.000000,0.000000,0.000000,0.00,0.058824,0.000000,0.058824,0.000000,0.058824,0.058824,0.000000,0.000000,0.058824,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824
1,Studio District,0.054054,0.000000,0.054054,0.027027,0.027027,0.027027,0.027027,0.000000,0.000000,0.000000,0.108108,0.000000,0.027027,0.027027,0.027027,0.081081,0.027027,0.000000,0.027027,0.000000,0.027027,0.000000,0.000000,0.000000,0.000000,0.000000,0.027027,0.000000,0.000000,0.000000,0.000000,0.000000,0.027027,0.000000,0.000000,0.000000,0.027027,0.00,0.027027,0.000000,0.054054,0.000000,0.000000,0.027027,0.000000,0.000000,0.027027,0.000000,0.027027,0.00,0.027027,0.000000,0.027027,0.000000,0.000000,0.000000,0.027027,0.027027,0.000000,0.000000,0.000000,0.000000,0.027027,0.000000,0.000000,0.000000,0.027027
2,The Beaches,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"The Beaches West,India Bazaar",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.045455,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.045455,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.00,0.045455,0.000000,0.045455,0.000000,0.000000,0.000000,0.045455,0.045455,0.000000,0.045455,0.000000,0.00,0.090909,0.045455,0.045455,0.045455,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.045455,0.000000,0.000000
4,"The Danforth West,Riverdale",0.022727,0.000000,0.022727,0.000000,0.000000,0.045455,0.022727,0.022727,0.000000,0.000000,0.022727,0.022727,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.022727,0.000000,0.022727,0.022727,0.000000,0.000000,0.000000,0.000000,0.000000,0.022727,0.045455,0.000000,0.000000,0.000000,0.181818,0.022727,0.000000,0.000000,0.00,0.068182,0.022727,0.045455,0.022727,0.022727,0.000000,0.000000,0.022727,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.022727,0.022727,0.000000,0.022727,0.000000,0.000000,0.000000,0.

In [89]:
# New Size after the Grouping
East_Toronto_grouped.shape

(5, 68)

In [90]:
num_top_venues = 5

for hood in East_Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = East_Toronto_grouped[East_Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.12
1         Yoga Studio  0.06
2          Restaurant  0.06
3       Auto Workshop  0.06
4       Garden Center  0.06


----Studio District----
                 venue  freq
0                 Café  0.11
1          Coffee Shop  0.08
2  American Restaurant  0.05
3               Bakery  0.05
4   Italian Restaurant  0.05


----The Beaches----
                       venue  freq
0                Music Venue  0.25
1          Health Food Store  0.25
2                        Pub  0.25
3  Latin American Restaurant  0.00
4                Music Store  0.00


----The Beaches West,India Bazaar----
                  venue  freq
0                  Park  0.09
1        Sandwich Place  0.09
2                   Pub  0.05
3  Fast Food Restaurant  0.05
4         Movie Theater  0.05


----The Danforth West,Riverdale----
                    venue  freq
0        Greek Restaurant  0.18
1             Coffee

### Put these common venues into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [91]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Create the new dataframe and display the top 10 venues for each of East Toronto's neighborhood.

In [92]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = East_Toronto_grouped['Neighborhood']

for ind in np.arange(East_Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(East_Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Brewery,Park,Burrito Place,Pizza Place,Farmers Market,Restaurant,Fast Food Restaurant,Recording Studio
1,Studio District,Café,Coffee Shop,American Restaurant,Italian Restaurant,Bakery,Bar,Bookstore,Brewery,Bank,Cheese Shop
2,The Beaches,Music Venue,Pub,Health Food Store,Diner,Comic Shop,Convenience Store,Cosmetics Shop,Coworking Space,Dessert Shop,Farmers Market
3,"The Beaches West,India Bazaar",Park,Sandwich Place,Food & Drink Shop,Pub,Italian Restaurant,Gym,Light Rail Station,Liquor Store,Fish & Chips Shop,Movie Theater
4,"The Danforth West,Riverdale",Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Furniture / Home Store,Fruit & Vegetable Store,Japanese Restaurant,Indian Restaurant,Grocery Store


## Cluster Neighborhoods of East Toronto - Show 3 Clusters

Run *k*-means to cluster the neighborhood into 3 clusters.

In [93]:
# set number of clusters
kclusters = 3

East_Toronto_grouped_clustering = East_Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(East_Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 1, 0, 2], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [94]:
# add clustering labels

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

East_Toronto_merged = East_Toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
East_Toronto_merged = East_Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

East_Toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Music Venue,Pub,Health Food Store,Diner,Comic Shop,Convenience Store,Cosmetics Shop,Coworking Space,Dessert Shop,Farmers Market
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,2,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Furniture / Home Store,Fruit & Vegetable Store,Japanese Restaurant,Indian Restaurant,Grocery Store
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,0,Park,Sandwich Place,Food & Drink Shop,Pub,Italian Restaurant,Gym,Light Rail Station,Liquor Store,Fish & Chips Shop,Movie Theater
3,M4M,East Toronto,Studio District,43.659526,-79.340923,2,Café,Coffee Shop,American Restaurant,Italian Restaurant,Bakery,Bar,Bookstore,Brewery,Bank,Cheese Shop
4,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,0,Light Rail Station,Yoga Studio,Brewery,Park,Burrito Place,Pizza Place,Farmers Market,Restaurant,Fast Food Restaurant,Recording Studio


In [95]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(East_Toronto_merged['Latitude'], East_Toronto_merged['Longitude'], East_Toronto_merged['Neighbourhood'], East_Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine East Toronto Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [96]:
East_Toronto_merged.loc[East_Toronto_merged['Cluster Labels'] == 0, East_Toronto_merged.columns[[1] + list(range(3, East_Toronto_merged.shape[1]))]]

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,East Toronto,43.668999,-79.315572,0,Park,Sandwich Place,Food & Drink Shop,Pub,Italian Restaurant,Gym,Light Rail Station,Liquor Store,Fish & Chips Shop,Movie Theater
4,East Toronto,43.662744,-79.321558,0,Light Rail Station,Yoga Studio,Brewery,Park,Burrito Place,Pizza Place,Farmers Market,Restaurant,Fast Food Restaurant,Recording Studio


#### Cluster 2

In [97]:
East_Toronto_merged.loc[East_Toronto_merged['Cluster Labels'] == 1, East_Toronto_merged.columns[[1] + list(range(3, East_Toronto_merged.shape[1]))]]

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,43.676357,-79.293031,1,Music Venue,Pub,Health Food Store,Diner,Comic Shop,Convenience Store,Cosmetics Shop,Coworking Space,Dessert Shop,Farmers Market


#### Cluster 3

In [98]:
East_Toronto_merged.loc[East_Toronto_merged['Cluster Labels'] == 2, East_Toronto_merged.columns[[1] + list(range(3, East_Toronto_merged.shape[1]))]]

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Toronto,43.679557,-79.352188,2,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Furniture / Home Store,Fruit & Vegetable Store,Japanese Restaurant,Indian Restaurant,Grocery Store
3,East Toronto,43.659526,-79.340923,2,Café,Coffee Shop,American Restaurant,Italian Restaurant,Bakery,Bar,Bookstore,Brewery,Bank,Cheese Shop


## End of Assignment